In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
import os
os.chdir('/content/gdrive/My Drive/msrf')

In [24]:
from diu_net import diu_net_model, dice_coef, iou, dice_coef_loss
from PIL import Image
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import load_model


# **Loading the Data**

In [25]:
image_list = [f[: -4] for f in os.listdir('train/patched_tissues/images')]
data_dir = 'train'
input_dir = 'train/patched_tissues/images'
target_dir = 'train/patched_masks/images'

x_array = np.zeros((len(image_list), 256, 256, 3), dtype = np.float32)
y_array = np.zeros((len(image_list), 256, 256), dtype = np.float32)

for (i, f) in enumerate(tqdm(image_list, desc = 'Loading Data...')):
  img = Image.open(os.path.join(input_dir, f + '.png'))
  msk = Image.open(os.path.join(target_dir, f + '.png'))

  x_array[i] = np.array(img, dtype = np.float32)
  y_array[i] = np.array(msk, dtype = np.float32)

print('Data Loaded!!')

Loading Data...: 100%|██████████| 592/592 [00:05<00:00, 113.40it/s]

Data Loaded!!


In [26]:
x_array.shape

(592, 256, 256, 3)

In [27]:
y_array.shape

(592, 256, 256)

In [28]:
model = diu_net_model((256, 256, 3), compile = False)

In [29]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = [dice_coef_loss], metrics = [dice_coef, iou])

In [34]:
model.load_weights('weights/w6.h5')

ValueError: ignored

In [ ]:
model.summary(expand_nested = True)

# **Generators**

# **Training**

In [ ]:
#model 3 did good after 30 and 45 epochs

In [ ]:
history = model.fit(x_array, y_array, batch_size = 12, epochs = 30, verbose = 1, validation_split = 0.1)
df = history.history

In [ ]:
#model.save_weights('weights/w7.h5')

In [ ]:
#model.save('/content/drive/MyDrive/msrf/models/model_001.h5')

In [ ]:
model.evaluate(x_array, y_array)

# **Patch Test**

In [ ]:
test = model.predict(np.expand_dims(np.array(Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Tissue Images/TCGA-A6-6782-01A-01-BS1.tif'))[257: 513, 257: 513], axis = 0))

In [ ]:
test = (test > 0.5) * 1

In [ ]:
plt.figure(figsize = (27, 9))
ax = plt.subplot(1, 3, 1)
plt.title('Input', fontsize = 30)
plt.imshow(np.array(Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Tissue Images/TCGA-A6-6782-01A-01-BS1.tif'))[257: 513, 257: 513]/255.)
ax = plt.subplot(1, 3, 2)
plt.title('Ground Truth', fontsize = 30)
plt.imshow(np.array(Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Binary_masks/TCGA-A6-6782-01A-01-BS1.png'))[257: 513, 257: 513])
ax = plt.subplot(1, 3, 3)
plt.title('Predicted Mask', fontsize = 30)
plt.imshow(np.squeeze(test[0], axis = -1))
plt.show()

In [ ]:
gt_test = np.array(Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Binary_masks/TCGA-A6-6782-01A-01-BS1.png'))[257: 513, 257: 513]
pred_test = np.squeeze(test[0], axis = -1)

In [ ]:
test_iou = (np.sum(np.multiply(gt_test, pred_test)))/(np.sum(gt_test) + np.sum(pred_test) - np.sum(np.multiply(gt_test, pred_test)))
test_dice = (2 * np.sum(np.multiply(gt_test, pred_test)))/(np.sum(gt_test) + np.sum(pred_test))

In [ ]:
print(f'Test Image DICE: {test_dice}')
print(f'Test Image IoU: {test_iou}')


# **Image Test**

In [ ]:
image = Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Tissue Images/TCGA-FG-A4MU-01B-01-TS1.tif').resize((1024, 1024), Image.NEAREST)

In [ ]:
ground_truth = Image.open('/content/gdrive/MyDrive/msrf/MoNuSegTestData/Binary_masks/TCGA-FG-A4MU-01B-01-TS1.png')

In [ ]:
image = np.array(image)

In [ ]:
def predict(image):
  temp = np.zeros((1024, 1024), dtype = np.uint8)
  for i in range(0, 769, 256):
    for j in range(0, 769, 256):
      temp[i: i + 256, j: j + 256] = np.reshape(((model.predict(np.expand_dims(image[i: i + 256, j: j + 256], axis = 0))>0.5) * 1), (256, 256))
  return Image.fromarray(temp).resize((1000, 1000))

In [ ]:
pred = predict(image)

In [ ]:
pred = np.array(pred)

In [ ]:
plt.figure(figsize = (27, 9))
ax = plt.subplot(1, 3, 1)
plt.title('Input', fontsize = 30)
plt.imshow(image)
ax = plt.subplot(1, 3, 2)
plt.title('Ground Truth', fontsize = 30)
plt.imshow(ground_truth)
ax = plt.subplot(1, 3, 3)
plt.title('Predicted Mask', fontsize = 30)
plt.imshow(pred)
plt.show()

In [ ]:
test_iou = (np.sum(np.multiply(ground_truth, pred)))/(np.sum(ground_truth) + np.sum(pred) - np.sum(np.multiply(ground_truth, pred)))
test_dice = (2 * np.sum(np.multiply(ground_truth, pred)))/(np.sum(ground_truth) + np.sum(pred))
print(f'Test Image DICE: {test_dice}')
print(f'Test Image IoU: {test_iou}')